# APPLIED DATA ANALYSIS PROJECT

In [2]:
import pandas as pd
import requests
import re

from bs4 import BeautifulSoup

In [3]:
URL_QS = 'https://en.wikipedia.org/wiki/List_of_former_European_colonies'
r = requests.get(URL_QS)
soup = BeautifulSoup(r.text, 'lxml')

In [4]:
colonizer_countries = []

# To find the name of all colonizer country of Europe, we take care about 2 things :
#  - If there is more than 2 spaces in the text -> isn't a country
#  - If there is less than 2 letters in the text -> isn't a country

for country in soup.findAll('p'):
    nb_space = sum(c.isspace() for c in country.text)
    
    if country.text not in colonizer_countries and nb_space < 3 and len(country.text) > 2:
        colonizer_countries.append(country.text)

# List Cleaning
colonizer_countries = [country.replace('\n', '') for country in colonizer_countries]

colonizer_countries

['Denmark',
 'France',
 'Netherlands',
 'Norway',
 'Portugal',
 'Russia',
 'Spain',
 'Sweden',
 'Brandenburg',
 'Britain',
 'Courland',
 'Denmark-Norway',
 'Knights of Malta',
 'Belgium',
 'Germany',
 'Italy',
 'Austria',
 'United Kingdom',
 'Germany',
 'Greece']

In [5]:
regions = []

for region in soup.find("ul").findAll('li'):
    regions.append(region.text[2:])

del regions[-1], regions[-1] 

regions

['North America',
 'West Indies and the Caribbean',
 'South America',
 'Africa',
 'Indian Ocean',
 'Middle East',
 'Indian Subcontinent',
 'Asia-Pacific',
 'Europe']

In [363]:
def get_colonizer(infobox, colonizer):
    
    new_colonizer = ""

    independance = infobox.find(text = re.compile("Independence"))
    
    #infobox.find(text = re.compile("Independence")).next.next.next.td.text.split(" ")[-1]

    
    if independance:
        independ_index = infobox.find(text = "Independence")
        if independ_index is None: independ_index = infobox.find(text = "Formation")
        if independ_index is None: return colonizer, 0
        
        new_colonizer = independ_index.find_next("a").text
        independ_date = independ_index.find_next("td").text.split()[-1]
        count = 0

        independ_index_tmp = independ_index

        while(new_colonizer not in colonizer_countries and count < 10):    
            independ_index_tmp = independ_index_tmp.find_next("a")

            new_colonizer = independ_index_tmp.text
            independ_date = independ_index_tmp.find_next("td").text.split()[-1]

            count += 1 

        if new_colonizer not in colonizer_countries:    
            exclu_list = ['•', 'from', 'the']
            cleaned = []

            for string in independ_index.find_next("tr").text.split():
                if string not in exclu_list:
                    cleaned.append(string)

            if cleaned[0] == "United": new_colonizer = cleaned[0] + " "+ cleaned[1]
            else: new_colonizer = cleaned[0]

            independ_date = cleaned[-1].split('[')[0] 

        if new_colonizer in colonizer_countries: 
            return new_colonizer, independ_date
        else:
            colonizer, 0

    return colonizer, 0

def get_infobox(soup):

    infobox = soup.find("table", class_="infobox geography")
    if not infobox: infobox = soup.find("table", class_="infobox geography vcard")
    if not infobox: infobox = soup.find("table", class_="infobox geography vcard vevent")
    if not infobox: infobox = soup.find("table", class_="infobox vcard ")
        
    return infobox

def get_ID(infobox):

    ID = infobox.find(text = re.compile("ISO 3166 code"))
    
    if ID is not None:
        return ID.next.next.text
    else:
        return False

def get_size(infobox):
    
    area = infobox.find("th", text="Area")
    
    if area:
        size = area.parent.next_sibling.next_sibling.td.text.split("\xa0")[0]

        # Size cleaning
        if '–' in size: size = size.split("–")[0]
        if '[' in size: size = size.split("[")[0]
        if ',' in size: size = size.replace(",", "")
        if '.' in size: size = size.split(".")[0]
        if ' ' in size: size = size.split(" ")[0]                                       
        if len(size) == 0: size = 0
            
        return int(size)
    
    else:
        return 0
    
def get_succeeded_by(infobox):
    
    succeeded_by = infobox.find("b", text="Succeeded by")
    
    if succeeded_by: 
        succeeded_by = succeeded_by.parent.parent.next_sibling.next_sibling

        new_country = succeeded_by.find(style="border:0; padding:0; vertical-align:middle; text-align:right;").text
        URL = succeeded_by.find(style="border:0; padding:0; vertical-align:middle; text-align:right;").a['href']

        return new_country, URL

    else:
        return False, False

In [364]:
def check_country(df, colonized, colonizer, URL, nb_check):
    
    # The checking of each colony is as follows :
    # - If a infobox (the resum on the right side) exist, we check the size. If the size is inferior than a 
    #   certain value, we does not considere this country because it is more a island or a city than a country.
    #   We also check if the colonizer country is the same at the end as the begin. And the end, we take the 
    #   colonizer country juste before the independance.
    # - If a word like 'disestablished' or 'disestablishments' is found in the page, we considere that the
    #   country don't exist any more. We check if a new country was created on find 
    #   the sentence "Succeeded by" in the infobox (if it extist) and repeat the checking with the new 
    #   country if found
    
    if nb_check > 5:
        return
    
    URL_QS = 'https://en.wikipedia.org' + URL
    r = requests.get(URL_QS)
    soup = BeautifulSoup(r.text, 'lxml')

    infobox = get_infobox(soup) 

    # Check if the crountry still exist and if the infobox exist
    if not soup(text=re.compile('disestablished')) and not soup(text=re.compile('disestablishments')): 
        if infobox:
            ID = get_ID(infobox)
            colonizer, independ_date = get_colonizer(infobox, colonizer)
            size = get_size(infobox)

            # On vérifie la taille, si < 1'500 km : osef des îles/villes
            # On vérifie aussi que le pays n'apparait pas déjà dans le datafram
            if size > 1500 and len(df[df['Colonized Country']==colonized]) < 1 and ID:
                df.loc[len(df)+1] = [colonized, ID, independ_date, colonizer, URL]  

    else:
        if infobox:
            new_country, URL = get_succeeded_by(infobox)
            
            if new_country:
                check_country(df, new_country, colonizer, URL, nb_check + 1)

In [365]:
# Dataframe creation
colonization_df = pd.DataFrame(columns=['Colonized Country', 'ID',' Date', 'Colonizer Country', "URL"])

URL_QS = 'https://en.wikipedia.org/wiki/List_of_former_European_colonies'
r = requests.get(URL_QS)
soup = BeautifulSoup(r.text, 'lxml')

#for colonizer_country in colonizer_countries:
colonizer_country = 'France'
print(colonizer_country, "...\n")

# Trouve tout les emplacements où on fait mention du pays "colonizer"
colonizer_index = soup.find_all("p", text=colonizer_country) 

for i in range (len(colonizer_index)):
    colonized_countries = colonizer_index[i].next_sibling.next_sibling('a')

    for colonized_country in colonized_countries:
        if colonized_country.text not in regions and len(colonized_country.attrs) < 3:
            print(colonized_country.text)
            check_country(colonization_df, colonized_country.text, colonizer_country, colonized_country['href'], 1)

colonization_df

France ...

New France
Île-Royale
Saint-Domingue
Anguilla
Antigua and Barbuda
Dominica
Grenada
Haiti
Montserrat
Saint Barthélemy
Nevis
Saint Christophe
Saint-Domingue
Haiti
Saint Vincent and the Grenadines
Sint Eustatius
Tobago
Rio de Janeiro
São Luís
Albreda
Dahomey
Benin
French Algeria
French Cameroons
French Chad
French Congo
French Guinea
Guinea
French Upper Volta
Republic of Upper Volta
Burkina Faso
French Somaliland
Djibouti
French Sudan
Mali
French Togoland
Togo
Gabon
Ivory Coast
Malagasy Protectorate
Colonial Mauritania
French protectorate in Morocco
Niger
Central African Republic
Senegambia and Niger
Upper Senegal and Niger
Colony of Niger
French protectorate of Tunisia
Isle de France
Mauritius
French Madagascar
French Seychelles
French Mandate for Syria and the Lebanon
Pondicherry
Yanaon
Zhanjiang
Guangdong
in Shanghai
Guangdong
Tianjin
Yunnan
Guangxi
Hainan
Guangdong
Indochina
Cambodia
Laos
Vietnam
Annam
Cochinchina
Tonkin
New Hebrides
Vanuatu


,Colonized Country,ID,Date,Colonizer Country,URL
1,Canada,CA,0,France,/wiki/Canada
2,Haiti,HT,1804,France,/wiki/Haiti
3,Benin,BJ,1960,France,/wiki/Benin
4,Cameroon,CM,0,France,/wiki/Cameroon
5,Chad,TD,1960,France,/wiki/Chad
6,Central African Republic,CF,1960,France,/wiki/Central_African_Republic
7,Guinea,GN,1958,France,/wiki/Guinea
8,Burkina Faso,BF,1960,France,/wiki/Burkina_Faso
9,Djibouti,DJ,1977,France,/wiki/Djibouti
10,Mali,ML,0,France,/wiki/Mali


In [ ]:
# TO DO
# - Pour les dates, faire encore le cas où y'a marqué directement "Independence from France"
# - Merge Britain & United Kingdom


In [359]:
URL_QS = 'https://en.wikipedia.org/wiki/Morocco'
r = requests.get(URL_QS)
soup = BeautifulSoup(r.text, 'lxml')

infobox = soup.find("table", class_="infobox geography")
if not infobox: infobox = soup.find("table", class_="infobox geography vcard")
if not infobox: infobox = soup.find("table", class_="infobox geography vcard vevent")
if not infobox: infobox = soup.find("table", class_="infobox vcard ")


independ_index = infobox.find(text = "Independence")
if independ_index is None: independ_index = infobox.find(text = "Formation")

if independ_index is not None:

    new_country = independ_index.find_next("a").text
    independ_date = independ_index.find_next("td").text.split()[-1]
    count = 0

    independ_index_tmp = independ_index

    while(new_country not in colonizer_countries and count < 10):    
        independ_index_tmp = independ_index_tmp.find_next("a")

        new_country = independ_index_tmp.text
        independ_date = independ_index_tmp.find_next("td").text.split()[-1]
        print(new_country)
        count += 1 

    if new_country not in colonizer_countries:    
        exclu_list = ['•', 'from', 'the']
        cleaned = []

        print(independ_index.find_next("tr").text)
        
        for string in independ_index.find_next("tr").text.split():
            if string not in exclu_list:
                cleaned.append(string)

        print(cleaned)
        if cleaned[0] == "United": new_country = cleaned[0] + cleaned[1]
        else: new_country = cleaned[0]

        independ_date = cleaned[-1].split('[')[0] 
    
print(new_country, independ_date)

Area
[b]
40th or 57th
Population
[5]
GDP
PPP
[6]
[7]
GDP

Area

['Area']
Area Area
